# ഓട്ടോജെൻ ഉപയോഗിച്ച് ഏജന്റിക് RAG

ഈ നോട്ട്ബുക്ക് മെച്ചപ്പെടുത്തിയ മൂല്യനിർണയ ശേഷികളോടെയുള്ള ഓട്ടോജെൻ ഏജന്റുകളെ ഉപയോഗിച്ച് Retrieval-Augmented Generation (RAG) നടപ്പിലാക്കുന്നത് പ്രദർശിപ്പിക്കുന്നു.


SQLite പതിപ്പ് പരിഹാരം  
നിങ്ങൾക്ക് താഴെ കാണുന്ന പിശക് നേരിടുന്നുവെങ്കിൽ:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```
  
നിങ്ങളുടെ നോട്ട്‌ബുക്കിന്റെ തുടക്കത്തിൽ ഈ കോഡ് ബ്ലോക്ക് അൺകമന്റ് ചെയ്യുക:  


In [2]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
!pip install chromadb

In [3]:
import os
import time
import asyncio
from typing import List, Dict
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient

import chromadb

load_dotenv()

True

## ക്ലയന്റ് സൃഷ്ടിക്കുക

ആദ്യം, Azure AI Chat Completion Client നമുക്ക് ആരംഭിക്കാം. ഉപയോക്തൃ ചോദ്യങ്ങൾക്ക് പ്രതികരണങ്ങൾ സൃഷ്ടിക്കാൻ Azure OpenAI സേവനവുമായി ഇടപഴകാൻ ഈ ക്ലയന്റ് ഉപയോഗിക്കും.


In [4]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

## വെക്ടർ ഡാറ്റാബേസ് ആരംഭിക്കൽ

നാം സ്ഥിരമായ സംഭരണത്തോടെ ChromaDB ആരംഭിക്കുകയും മെച്ചപ്പെടുത്തിയ സാമ്പിള്‍ ഡോക്യുമെന്റുകള്‍ ചേര്‍ക്കുകയും ചെയ്യുന്നു. ChromaDB ഡോക്യുമെന്റുകള്‍ സംഭരിക്കുകയും, കൃത്യമായ മറുപടികള്‍ സൃഷ്ടിക്കുന്നതിന് പ്രാസംഗികത നല്‍കുകയും ചെയ്യുന്നതിനായി ഉപയോഗിക്കും.


In [5]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# Enhanced sample documents
documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours."
]

# Add documents with metadata
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

## കോൺടെക്സ്റ്റ് പ്രൊവൈഡർ ഇംപ്ലിമെന്റേഷൻ

`ContextProvider` ക്ലാസ് വിവിധ ഉറവിടങ്ങളിൽ നിന്ന് കോൺടെക്സ്റ്റ് തിരയലും സംയോജനവും കൈകാര്യം ചെയ്യുന്നു:

1. **വെക്ടർ ഡാറ്റാബേസ് തിരയൽ**: ChromaDB ഉപയോഗിച്ച് യാത്രാ രേഖകളിൽ സെമാന്റിക് തിരയൽ നടത്തുന്നു
2. **കാലാവസ്ഥാ വിവരങ്ങൾ**: പ്രധാന നഗരങ്ങൾക്ക് സിമുലേറ്റഡ് കാലാവസ്ഥാ ഡാറ്റാബേസ് നിലനിർത്തുന്നു
3. **ഏകീകൃത കോൺടെക്സ്റ്റ്**: രേഖകളും കാലാവസ്ഥാ ഡാറ്റയും സംയോജിപ്പിച്ച് സമഗ്രമായ കോൺടെക്സ്റ്റ് സൃഷ്ടിക്കുന്നു

പ്രധാനമായ രീതികൾ:
- `get_retrieval_context()`: ക്വറിയുടെ അടിസ്ഥാനത്തിൽ അനുയോജ്യമായ രേഖകൾ തിരയുന്നു
- `get_weather_data()`: നിർദ്ദിഷ്ട സ്ഥലങ്ങളിലെ കാലാവസ്ഥാ വിവരങ്ങൾ നൽകുന്നു
- `get_unified_context()`: രേഖാ കോൺടെക്സ്റ്റും കാലാവസ്ഥാ കോൺടെക്സ്റ്റും സംയോജിപ്പിച്ച് മെച്ചപ്പെട്ട പ്രതികരണങ്ങൾക്കായി ഉപയോഗിക്കുന്നു


In [6]:
class ContextProvider:
    def __init__(self, collection):
        self.collection = collection
        # Simulated weather database
        self.weather_database = {
            "new york": {"temperature": 72, "condition": "Partly Cloudy", "humidity": 65, "wind": "10 mph"},
            "london": {"temperature": 60, "condition": "Rainy", "humidity": 80, "wind": "15 mph"},
            "tokyo": {"temperature": 75, "condition": "Sunny", "humidity": 50, "wind": "5 mph"},
            "sydney": {"temperature": 80, "condition": "Clear", "humidity": 45, "wind": "12 mph"},
            "paris": {"temperature": 68, "condition": "Cloudy", "humidity": 70, "wind": "8 mph"},
        }
    
    def get_retrieval_context(self, query: str) -> str:
        """Retrieves relevant documents from vector database based on query."""
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_strings = []
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_strings.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_strings) if context_strings else "No relevant documents found"
    
    def get_weather_data(self, location: str) -> str:
        """Simulates retrieving weather data for a given location."""
        if not location:
            return ""
            
        location_key = location.lower()
        if location_key in self.weather_database:
            data = self.weather_database[location_key]
            return f"Weather for {location.title()}:\n" \
                   f"Temperature: {data['temperature']}°F\n" \
                   f"Condition: {data['condition']}\n" \
                   f"Humidity: {data['humidity']}%\n" \
                   f"Wind: {data['wind']}"
        else:
            return f"No weather data available for {location}."
    
    def get_unified_context(self, query: str, location: str = None) -> str:
        """Returns a unified context combining both document retrieval and weather data."""
        retrieval_context = self.get_retrieval_context(query)
        
        weather_context = ""
        if location:
            weather_context = self.get_weather_data(location)
            weather_intro = f"\nWeather Information for {location}:\n"
        else:
            weather_intro = ""
        
        return f"Retrieved Context:\n{retrieval_context}\n\n{weather_intro}{weather_context}"

## ഏജന്റ് കോൺഫിഗറേഷൻ

നാം റിട്രീവൽ ഏജന്റും അസിസ്റ്റന്റ് ഏജന്റും കോൺഫിഗർ ചെയ്യുന്നു. സെമാന്റിക് സേർച്ച് ഉപയോഗിച്ച് അനുയോജ്യമായ വിവരങ്ങൾ കണ്ടെത്തുന്നതിൽ റിട്രീവൽ ഏജന്റ് പ്രത്യേകതയുള്ളതാണ്, അതേസമയം അസിസ്റ്റന്റ് റിട്രീവ് ചെയ്ത വിവരങ്ങളുടെ അടിസ്ഥാനത്തിൽ വിശദമായ പ്രതികരണങ്ങൾ സൃഷ്ടിക്കുന്നു.


In [7]:
# Create agents with enhanced capabilities
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message=(
        "You are a helpful AI assistant that provides answers using ONLY the provided context. "
        "Do NOT include any external information. Base your answer entirely on the context given below."
    ),
)

## RAG ഉപയോഗിച്ച് ക്വറി പ്രോസസ്സിംഗ്

ക്വറി അസിസ്റ്റന്റിലേക്ക് അയക്കാനും, പ്രതികരണം പ്രോസസ്സ് ചെയ്യാനും, അത് വിലയിരുത്താനും `ask_rag` ഫംഗ്ഷൻ നമുക്ക് നിർവചിക്കുന്നു. ഈ ഫംഗ്ഷൻ അസിസ്റ്റന്റുമായി സംവാദം കൈകാര്യം ചെയ്യുകയും പ്രതികരണത്തിന്റെ ഗുണനിലവാരം അളക്കാൻ എവാലുവേറ്റർ ഉപയോഗിക്കുകയും ചെയ്യുന്നു.


In [8]:
async def ask_rag_agent(query: str, context_provider: ContextProvider, location: str = None):
    """
    Sends a query to the assistant agent with context from the provider.
    
    Args:
        query: The user's question
        context_provider: The context provider instance
        location: Optional location for weather queries
    """
    try:
        # Get unified context
        context = context_provider.get_unified_context(query, location)
        
        # Augment the query with context
        augmented_query = (
            f"{context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide a helpful answer."
        )

        # Send the augmented query to the assistant
        start_time = time.time()
        response = await assistant.on_messages(
            [TextMessage(content=augmented_query, source="user")],
            cancellation_token=CancellationToken(),
        )
        processing_time = time.time() - start_time
        
        return {
            'query': query,
            'response': response.chat_message.content,
            'processing_time': processing_time,
            'location': location
        }
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

# ഉദാഹരണ ഉപയോഗം

നാം evaluator ആരംഭിച്ച് പ്രോസസ്സ് ചെയ്യാനും വിലയിരുത്താനും ആഗ്രഹിക്കുന്ന queries നിർവചിക്കുന്നു.


In [ ]:
async def main():
    # Initialize context provider
    context_provider = ContextProvider(collection)
    
    # Example queries
    queries = [
        {"query": "What does Contoso's travel insurance cover?"},
        {"query": "What's the weather like in London?", "location": "london"},
        {"query": "What luxury destinations does Contoso offer and what's the weather in Paris?", "location": "paris"},
    ]
    
    print("=== Autogen RAG Demo ===")
    for query_data in queries:
        query = query_data["query"]
        location = query_data.get("location")
        
        print(f"\n\nQuery: {query}")
        if location:
            print(f"Location: {location}")
        
        # Show the context being used
        context = context_provider.get_unified_context(query, location)
        print("\n--- Context Used ---")
        print(context)
        print("-------------------")
        
        # Get response from the agent
        result = await ask_rag_agent(query, context_provider, location)
        if result:
            print(f"\nResponse: {result['response']}")
        print("\n" + "="*50)

## സ്ക്രിപ്റ്റ് പ്രവർത്തിപ്പിക്കുക

സ്ക്രിപ്റ്റ് ഇന്ററാക്ടീവ് പരിസ്ഥിതിയിൽ പ്രവർത്തിക്കുന്നുണ്ടോ അല്ലെങ്കിൽ സാധാരണ സ്ക്രിപ്റ്റാണോ എന്ന് പരിശോധിച്ച്, പ്രധാന ഫംഗ്ഷൻ അനുയോജ്യമായി പ്രവർത്തിപ്പിക്കുന്നു.


In [12]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        await main()
    else:
        asyncio.run(main())

=== Autogen RAG Demo ===


Query: What does Contoso's travel insurance cover?

--- Context Used ---
Retrieved Context:
Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


-------------------

Response: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.



Query: What's the weather like in London?
Location: london

--- Context Used ---
Retrieved Context:
Document: Popular destinations include the Maldives, Swiss Alps, and African safaris.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


Weather Information for london:
Weather for London:
T

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസത്യവാദം**:  
ഈ രേഖ AI വിവർത്തന സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. ഞങ്ങൾ കൃത്യതയ്ക്കായി ശ്രമിക്കുന്നുവെങ്കിലും, ഓട്ടോമേറ്റഡ് വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റായ വിവരങ്ങൾ ഉണ്ടാകാൻ സാധ്യതയുണ്ട്. അതിന്റെ മാതൃഭാഷയിലുള്ള മൗലികരേഖയാണ് വിശ്വസനീയമായ ഉറവിടമായി കണക്കാക്കേണ്ടത്. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യുന്നു. ഈ വിവർത്തനം ഉപയോഗിച്ച് ഉണ്ടാകുന്ന തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
